In [1]:
!python --version

Python 3.11.4


In [2]:
%pip install tensorflow numpy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tensorflow as tf
import numpy as np
import time
import json

# https://github.com/tensorflow/tensorflow/issues/62075

2025-05-27 21:37:59.692067: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-27 21:37:59.692541: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-27 21:37:59.695761: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-27 21:37:59.703064: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748374679.716807   98184 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748374679.72

In [4]:
file_path = "../../data/embeddings/imdb_dataset_prepared.json"

with open(file_path, 'r', encoding='utf-8') as f:
    data_from_json = json.load(f)

X_train = np.array(data_from_json["X_train"])
y_train = np.array(data_from_json["y_train"])
X_test = np.array(data_from_json["X_test"])
y_test = np.array(data_from_json["y_test"])
emb = np.array(data_from_json["embeddings"])
vocab = data_from_json["vocab"]

X_train = X_train - 1 # indexy od zera
X_test  = X_test - 1

In [5]:
sequence_length = X_train.shape[1] # 130

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(sequence_length,)),
    tf.keras.layers.Embedding(input_dim=emb.shape[0], output_dim=emb.shape[1], weights=[emb]),
    tf.keras.layers.Conv1D(filters=8, kernel_size=3, activation="relu"),
    tf.keras.layers.MaxPooling1D(pool_size=8),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation="sigmoid"),
])

model.summary()

2025-05-27 21:38:02.025290: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 130, 50)        │       642,450 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 128, 8)         │         1,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 16, 8)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 643,787 (2.46 MB)

 Trainable params: 643,787 (2.46 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
dataset = (
    tf.data.Dataset
      .from_tensor_slices((X_train, y_train))
      .shuffle(buffer_size=len(X_train))
      .batch(64)
)

loss = lambda m, x, y: tf.keras.losses.binary_crossentropy(
    y,
    tf.squeeze(m(x), axis=-1)   # z (batch,1) → (batch,)
)
accuracy = lambda m, x, y: tf.reduce_mean(
    tf.keras.metrics.binary_accuracy(
        y,
        tf.squeeze(m(x), axis=-1)
    )
)

opt = tf.keras.optimizers.Adam()

epochs = 5
for epoch in range(1, epochs + 1):
    total_loss = 0.0
    total_acc = 0.0
    num_samples = 0

    t0 = time.time()
    for x, y in dataset:
        with tf.GradientTape() as tape:
            l = loss(model, x, y)
        grads = tape.gradient(l, model.trainable_weights)
        opt.apply_gradients(zip(grads, model.trainable_weights))

        total_loss += float(l)
        total_acc += float(accuracy(model, x, y))
        num_samples += 1
    t = time.time() - t0

    train_loss = total_loss / num_samples
    train_acc = total_acc / num_samples

    test_loss = float(loss(model, tf.convert_to_tensor(X_test), tf.convert_to_tensor(y_test)))
    test_acc = float(accuracy(model, tf.convert_to_tensor(X_test), tf.convert_to_tensor(y_test)))

    print(
        f"Epoch: {epoch} ({t:.2f}s)   "
        f"Train: (l: {train_loss:.2f}, a: {train_acc:.2f})   "
        f"Test:  (l: {test_loss:.2f}, a: {test_acc:.2f})"
    )

# https://github.com/tensorflow/tensorflow/issues/68593

2025-05-27 21:38:27.181240: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch: 1 (24.94s)   Train: (l: 0.54, a: 0.72)   Test:  (l: 0.40, a: 0.82)


2025-05-27 21:38:54.152519: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch: 2 (26.76s)   Train: (l: 0.34, a: 0.86)   Test:  (l: 0.33, a: 0.86)
Epoch: 3 (23.41s)   Train: (l: 0.26, a: 0.90)   Test:  (l: 0.31, a: 0.87)


2025-05-27 21:39:41.232920: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch: 4 (23.30s)   Train: (l: 0.20, a: 0.93)   Test:  (l: 0.32, a: 0.87)
Epoch: 5 (23.24s)   Train: (l: 0.14, a: 0.96)   Test:  (l: 0.35, a: 0.87)
